In [1]:
from google.colab import drive

import functools
import os

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

tf.enable_eager_execution()

drive.mount('/content/drive', force_remount=False)
%cd /content/drive/My\ Drive/ETH/Projects/Google/source
!ls
from constants import Constants as C
from dataset_strokes import TFRecordDiagramDataset
TF_DATA_PATH = "/content/drive/My Drive/ETH/Projects/Google/data/diagrams_with_strokes/test/diagrams_ramer_0.025.tf_record-?????-of-?????"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/ETH/Projects/Google/source
config.py     dataset_strokes.py   model_utils.py      utils.py
constants.py  model_components.py  training_eager.py
dataset.py    model.py		   training_static.py


In [2]:
batch_size = 8
train_data = TFRecordDiagramDataset(data_path=TF_DATA_PATH, meta_data_path=None, batch_size=batch_size, preprocessing=False, normalize=False, shuffle=False)
data_iter = train_data.get_iterator()
input_batch, target_batch = next(data_iter)

/content/drive/My Drive/ETH/Projects/Google/data/diagrams_with_strokes/test/diagrams_ramer_0.025.tf_record-?????-of-?????
Meta-data not found.
Skipping statistics...
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
sample_idx = 3
seq_len = input_batch[C.BATCH_SEQ_LEN][sample_idx].numpy()
vis_sample = input_batch["encoder_inputs"][sample_idx, 0:seq_len].numpy()
vis_sample[:, 1] = -1*vis_sample[:, 1]
pen = vis_sample[:, 2]
penups = np.where(pen == 1)[0]+1
strokes = np.split(vis_sample, penups, axis=0)
all_strokes = vis_sample

In [4]:
# First set up the figure, the axes, and the plot element
fig, ax = plt.subplots()
plt.close()
ax.set_xlim((all_strokes[:, 0].min(), all_strokes[:, 0].max()))
ax.set_ylim((all_strokes[:, 1].min(), all_strokes[:, 1].max()))
ax.set_facecolor((1.0, 1.0, 1.0))

lines = []
line_borders = []  # Start and end index of a stroke in the entire drawing.
current_len = 0
for index, stroke in enumerate(strokes):
    line_borders.append((current_len, current_len+stroke.shape[0]))
    lines.append(ax.plot([],[],lw=2)[0])
    current_len += stroke.shape[0] + 1

def init():
    for line in lines:
        line.set_data([],[])
    return lines
  
# animation function
def animate(i):
  for idx, (line, (line_start, line_end)) in enumerate(zip(lines, line_borders)):
    if i > line_end:
      # All strokes that have been visualized.
      line.set_data(strokes[idx][:, 0], strokes[idx][:, 1])
    else:
      line.set_data(strokes[idx][0:(i-line_start), 0], strokes[idx][0:(i-line_start), 1])
      break
  return lines

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=all_strokes.shape[0], interval=100, blit=True)
rc('animation', html='jshtml', embed_limit=128)
anim

# from IPython.display import HTML
# HTML(anim.to_html5_video(embed_limit=128))